# Testing possible atom mappings in perses
### Module to test how a hybrid system will be generated for a system

In [3]:
# from perses.utils.openeye import createOEMolFromSDF
from perses.utils.smallmolecules import render_atom_mapping
from perses.rjmc.topology_proposal import AtomMapper
from openeye import oechem
import itertools

In [12]:
##### molecules to play with

smiles = {'benzene':'c1ccccc1','phenol':'c1ccc(cc1)O','name':'c1ccc(cc1)O','cyclohexane':'C1CCCCC1'}

list_of_mols = []
for smi in smiles.values():
    print(smi)
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, smi)
    list_of_mols.append(mol)
    
print(f'{len(list_of_mols)} oemols have been generated')

c1ccccc1
c1ccc(cc1)O
c1ccc(cc1)O
C1CCCCC1
4 oemols have been generated


Starting by just using the three default options in perses

In [20]:
names_and_oemols = zip(smiles.keys(),list_of_mols)

map_strength = 'default'
for (nameA, molA),(nameB, molB) in itertools.combinations(names_and_oemols,2):
    Mapper = AtomMapper([molA,molB],map_strength=map_strength)
    render_atom_mapping(f'{nameA}to{nameB}_{map_strength}.png', Mapper.current_molecule, Mapper.proposed_molecule, Mapper.atom_map)



INFO:proposal_generator:Cannot generate atom map without breaking rings, trying again with weaker mapping.


NameError: name 'oegraphmol_proposed' is not defined